# Covid 19 Topic Analysis


In [0]:
#### cleaning up forieng articles
import pandas as pd 
import numpy as np
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
!pip install pyLDAvis
master = pd.read_csv('/content/drive/My Drive/covid 19 exploration/df/master_text.csv')
master
print(f"total Foreign articles dropes : {sum(master.main_text.str.contains(r' de | une | di '))}")

new_master = master[~master.main_text.str.contains(r' de | une | di | de | del | con | von | der ')]  #### previous runs should that i need to drop foereign articles
new_master

## Creating Bag of Words

In [2]:


from nltk import word_tokenize 
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
import sys
word_list = new_master.main_text.to_list()#[1:5]
stop_words=set(stopwords.words('english'))

counter=0
nreviews_list = []
for i,review in enumerate(word_list):
    # split into tokens by white space
    tokens = review.split()

    # remove remaining tokens that are not alphabetic
    tokens = [word.lower() for word in tokens if word.isalpha()]

    # filter out stop words
    tokens = [w for w in tokens if not w in stop_words]

    ntokens = ' '.join(tokens)
    nreviews_list.append(ntokens)
    
    counter+=1
    if counter%1000 == 0: 
      print(f"interation {i+1} of {new_master.shape[0]}")


docs = [word_tokenize(doc) for doc in nreviews_list]
lam=WordNetLemmatizer()
 
tokens =[[lam.lemmatize(token) for token in doc]for doc in docs]

#### bag of word freq
from gensim.corpora import Dictionary
dictionary = Dictionary(tokens)
dictionary.filter_extremes(no_below=10, no_above = .50, keep_n=100000 )
corpus = [dictionary.doc2bow(doc) for doc in docs]

##### bag of wrod TFIDF
from gensim import corpora, models
tfidf = models.TfidfModel(corpus)
corpus_tfidf = tfidf[corpus]

interation 1000 of 23903
interation 2000 of 23903
interation 3000 of 23903
interation 4000 of 23903
interation 5000 of 23903
interation 6000 of 23903
interation 7000 of 23903
interation 8000 of 23903
interation 9000 of 23903
interation 10000 of 23903
interation 11000 of 23903
interation 12000 of 23903
interation 13000 of 23903
interation 14000 of 23903
interation 15000 of 23903
interation 16000 of 23903
interation 17000 of 23903
interation 18000 of 23903
interation 19000 of 23903
interation 20000 of 23903
interation 21000 of 23903
interation 22000 of 23903
interation 23000 of 23903


## Training LDA Model

In [0]:
# Train LDA model.
from gensim.models import LdaMulticore
import multiprocessing
pool = multiprocessing.Pool()
pool._processes
# Set training parameters.
wrokers= pool._processes - 1   ### for setting the number of workers
num_topics = 15 ### should be between 5 to 20 hueristic
chunksize = 1000
passes = 10  
iterations = 10 
eval_every = None  # Don't evaluate model perplexity, takes too much time.

# Make a index to word dictionary.
temp = dictionary[0]  # This is only to "load" the dictionary.
id2word = dictionary.id2token

model = LdaMulticore(
    corpus=corpus,
    id2word=id2word,
    chunksize=chunksize,
    iterations=iterations,
    num_topics=num_topics,
    passes=passes,
    eval_every=eval_every, workers=wrokers
)

## Vizsualizing the Results

In [4]:
# Visualize the topics
import pyLDAvis
import pyLDAvis.gensim
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(model, corpus, dictionary=dictionary)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.192456  0.096784       1        1  10.719414
0      0.180175 -0.059080       2        1   8.095455
8     -0.195572 -0.089065       3        1   7.897550
14     0.100564 -0.200577       4        1   7.860724
4     -0.130226 -0.066288       5        1   7.372089
12    -0.029043 -0.132243       6        1   7.318099
7      0.189552  0.022224       7        1   7.216479
6     -0.218930  0.027686       8        1   6.330403
2      0.114124 -0.162505       9        1   6.157478
13     0.049837  0.206017      10        1   5.488556
11     0.075916  0.173028      11        1   5.467576
9     -0.064052  0.019544      12        1   5.413226
10     0.006091  0.005189      13        1   5.223634
5     -0.083460  0.099686      14        1   4.864638
3     -0.187432  0.059600      15        1   4.574677, topic_info=            Term           Freq          Total Category  logprob  loglift
494          rna   84558.000000   84558.000000  Default  30.0000  30.0000
53        health  100888.000000  100888.000000  Default  29.0000  29.0000
734       immune   63147.000000   63147.000000  Default  28.0000  28.0000
1439  expression   80328.000000   80328.000000  Default  27.0000  27.0000
563      vaccine   56070.000000   56070.000000  Default  26.0000  26.0000
...          ...            ...            ...      ...      ...      ...
479       region    5049.278809   30179.587891  Topic15  -5.5944   1.2967
133         site    4771.086914   25512.464844  Topic15  -5.6511   1.4080
1359    cellular    4746.684082   31698.160156  Topic15  -5.6562   1.1858
2621       vitro    4492.483887   22011.898438  Topic15  -5.7113   1.4955
1802      signal    4159.593750   16909.402344  Topic15  -5.7883   1.6822

[1335 rows x 6 columns], token_table=       Topic      Freq    Term
term                          
586        1  0.651045  access
586        2  0.018114  access
586        3  0.007351  access
586        4  0.048653  access
586        5  0.046378  access
...      ...       ...     ...
21469      9  0.001114       ϫ
21469     11  0.037868       ϫ
21469     12  0.014850       ϫ
21469     13  0.001485       ϫ
21469     14  0.012623       ϫ

[5544 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 1, 9, 15, 5, 13, 8, 7, 3, 14, 12, 10, 11, 6, 4])

In [17]:
# 
from gensim.models import CoherenceModel
CoherenceModel(model, corpus=corpus, tokens)

SyntaxError: ignored

In [29]:
# Compute Coherence Score using c_v
coherence_model_lda = CoherenceModel(model=model, texts=tokens[1:1000], dictionary=dictionary, coherence='c_v')
coherence_lda = coherence_model_lda.get_coherence()
print('\nCoherence Score: ', coherence_lda)


Coherence Score:  0.560538175619534
